In [1]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget import neurons
from voltagebudget import plc
from voltagebudget.util import locate_firsts
from voltagebudget.util import locate_peaks
from voltagebudget.util import filter_voltages

# -
def create_traces(ts_x, changes, fn):
    d_variances = []
    errors = []
    for c in changes:
        initial, target, obs, ts_opt = fn(ts_x, c)

#         d_v = initial - obs 
        d_v = obs 
        err = util.mae(ts_x, ts_opt)

        d_variances.append(d_v)
        errors.append(err)

    d_variances = np.asarray(d_variances)
    errors = np.asarray(errors)

    return d_variances, errors

Loading BokehJS ...

/Users/type/anaconda/envs/py2/lib/python2.7/site-packages/brian2/core/variables.py:174: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return np.issubdtype(np.bool, self.dtype)


In [2]:
t=0.5
stim_number=40
stim_onset=0.2
stim_offset=0.250
stim_rate=8
seed_stim=7525
time_step=1e-5

ns_x, ts_x = util.poisson_impulse(
        t,
        stim_onset,
        stim_offset - stim_onset,
        stim_rate,
        dt=time_step,
        n=stim_number,
        seed=seed_stim)

In [21]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Shared params

In [71]:
percent_change = 0.5
changes = np.linspace(0.0, percent_change, 100)

# Uniform

In [72]:
initial, target, obs, ts_opt = plc.uniform(ts_x, percent_change)

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [73]:
# -
d_variances_u, errors_u = create_traces(ts_x, changes, plc.uniform)

p = figure(plot_width=400, plot_height=200)
p.line(x=d_variances_u, y=errors_u, color="black")
p.xaxis.axis_label = 'Delta Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Extreme deviants

## Max

In [74]:
initial, target, obs, ts_opt = plc.max_deviant(ts_x, percent_change)

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

print(initial, target, obs)

# Save for fig
X = np.vstack([ns_x, ts_x, ts_opt])
header = "n,ts,ts_opt"
np.savetxt("../analysis/plc_spikes.csv", X.T, delimiter=",", header=header, comments="")

(0.012855946336618894, 0.006427973168309447, 0.006427551274891212)


In [75]:
# -
d_variances_ma, errors_ma = create_traces(ts_x, changes, plc.max_deviant)

p = figure(plot_width=300, plot_height=200)
p.line(x=d_variances_ma, y=errors_ma, color="black")
p.circle(x=d_variances_ma, y=errors_ma, color="black")
p.xaxis.axis_label = 'Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Save for fig
X = np.vstack([d_variances_ma, errors_ma, changes])
header = "variances,errors,percent_changes"
np.savetxt("../analysis/plc_traces.csv", X.T, delimiter=",", header=header, comments="")

## Left side deviants

In [76]:
initial, target, obs, ts_opt = plc.max_deviant(ts_x, percent_change, side='left')

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

print(initial, target, obs)

# Save for fig
X = np.vstack([ns_x, ts_x, ts_opt])
header = "n,ts,ts_opt"
np.savetxt("../analysis/plc_spikes.csv", X.T, delimiter=",", header=header, comments="")

(0.012855946336618894, 0.006427973168309447, 0.006423307796509974)


In [77]:
# -
d_variances_left, errors_left = create_traces(ts_x, changes, 
                                          lambda x, p: plc.max_deviant(x, p, side='left'))

p = figure(plot_width=300, plot_height=200)
p.line(x=d_variances_ma, y=errors_ma, color="black")
p.circle(x=d_variances_ma, y=errors_ma, color="black")
p.xaxis.axis_label = 'Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Save for fig
X = np.vstack([d_variances_ma, errors_ma, changes])
header = "variances,errors,percent_changes"
np.savetxt("../analysis/plc_traces.csv", X.T, delimiter=",", header=header, comments="")

# Compare traces

In [78]:
# -
p = figure(plot_width=500, plot_height=500)
p.line(x=d_variances_ma, y=errors_ma, color="black", legend="Max")
p.line(x=d_variances_left, y=errors_left, color="red", legend="Left")
p.line(x=d_variances_u, y=errors_u, color="orange", legend="Unif")
p.xaxis.axis_label = 'Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# CC

In [12]:
percent_change = .4

initial, target, obs, ts_opt = plc.coincidence(ts_x, percent_change)

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [13]:
initial, target, obs

(0.012855946336618894, 0.007713567801971336, 0.004145474460953744)

In [14]:
# -
d_variances_cc, errors_cc = create_traces(ts_x, changes, plc.coincidence)

p = figure(plot_width=400, plot_height=200)
p.line(x=d_variances_cc, y=errors_cc, color="black")
p.circle(x=d_variances_cc, y=errors_cc, color="black")
p.xaxis.axis_label = 'Delta Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [15]:
errors_cc

array([0.        , 0.01100377, 0.01100377, 0.01100377, 0.01100377,
       0.01100377, 0.01100377, 0.01100377, 0.01100377, 0.01100377,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01442502])